### cepik api

Documentation at: APi Cepik: https://api.cepik.gov.pl/doc
Script allows do grab data about vehicle registeret in Poland from CEPKI database.

Cepki contanins information from 2010 and is update weekly
This example downloads information about electric cars.

In [1]:
# required liaberies 
import pandas as pd
import numpy as np
import json
import time
import requests
import os
from datetime import datetime

In [2]:
#now_file = datetime.now()
#now_file = "generated_" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
#print (now_file)


#### Required filters - wojwodztwo and year (region and year)

In [3]:
# just to have it for copy/paste 
#wojew = ["02","04","06","08","10","12","14","16","18","20","22","24","26","28","30","32","XX"]
#years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022"]

# define wojewodztwo (region) and years
# api reguires during scraping years (maximum 2 years) and wojewodztwo - both parameters are required
wojew = ["02","04","06","08","10","12","14","16","18","20","22","24","26","28","30","32","XX"]
years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022"]

In [4]:
#Based url
based_url = "https://api.cepik.gov.pl/pojazdy?&limit=500" 

# Filters - juest to have it for futrer copy-paste
#filtr = '&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA'
# info how to build filter - check in documentation https://api.cepik.gov.pl/doc 
filtr = '&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'


upload_path = "J:/Dokumenty/Tomek/YT/samochody/cepik/grab/" # =define it in your machine
current_dic = "2022-06-05/" # Define a folder
base_name_xls = "Cepik_" #name of the xls/csv file
path_to_merge = upload_path + current_dic + "all/"
all_merged_data_file_name ="Cepik_all_data_in_one_file.csv" # name of the file with all data



In [5]:
try:
    del cars
    del df
except Exception:
    print("no df to del")

no df to del


In [6]:
try: 
    os.makedirs (path_to_merge)
    #print (upload_path + upload_path , "all/", " successfully created")
except Exception:
    print("dic: ", path_to_save, "aready exit" )

### Lets dig some data


In [7]:
# Wojewodztwo loop
for w in wojew:

    print ("-"*40)
    print("wojedzództwo:", w)
    
    wojewodz = "&wojewodztwo=" + w
    
    # Year loop
    print ("First year to dig:", years[0], "Last year to dig:", years[-1] )
    
    for y in years:
        
        # Make a directory 
        #path = os.path.join(upload_path, w , y) 
        #print ("y=",y)
        path_to_save = upload_path + current_dic + "/woj_" + w + "/" + y + "/" 
        
        try: 
            os.makedirs (path_to_save)
            print (path_to_save, " successfully created")
        except Exception:
            print("dic: ", path_to_save, "aready exit" )
        #print ("path_to_save", path_to_save)
        print ("-"*20)
        print("Year:", y)
        
        period = "&data-od=" + y + "0101&data-do=" + y + "1231"
                        
        # remove rec_no - rec_no is json file page number - limited to 500 results - its api limitation        
        try:
            del rec_no
        except Exception:
            print("no data to del")
       
        # setting up values
        rec_no = 0
        step = 1
        finished = False 
        number_of_waits = 0
        
        while not finished:
            
            page = "&page=" + str(step)
            url = based_url + period +  filtr + wojewodz + page
            print ("We dig in:", url)

            # requesting json api
            #------------------------------------    
            data_temp = requests.get(url) 
            #------------------------------------
            
            # checking the response - 200 is correct
            if data_temp.status_code != 200:
                number_of_waits = number_of_waits + 1
                print ("digging status NOK: ", data_temp.status_code)
                print ("wait for 5 sek.", number_of_waits , "times")
                time.sleep(5) # just wait 5 sek
            else:
                # status code is 200 so lets take the data
                print ("digging OK -", "data statatus code:",data_temp.status_code)
                data = data_temp.json()

                # extracting the section of json file i need
                df = pd.json_normalize(data, record_path =['data'])
                
                # how many times we need to iterate data to download
                rec_no = pd.json_normalize(data)
                rec_no = rec_no["meta.count"][0]
                records_tc =  rec_no
                rec_no = round(rec_no/500+1,0)

                print ("page/step: ",step," Records in woj: ",w, "records to download:",records_tc ," pages to download: ", rec_no)
                                
                # ----------- aggregtion data into the year and woj ----------------
                # if first step lets make cars as a df
                if step == 1:
                    cars_in_years_and_woj = df
                else:
                    # add another set of data form steps in the common df
                    cars_in_years_and_woj = cars_in_years_and_woj.append(df)
                
                # ----------- aggregtion data into the year and woj ----------------
                
                # exception in case of to many data may be required but I did not write it here
                # save each set of data from particular step steps into the csv
                
                # Definition of the file name to save
                now_file = "generated_" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
                name_xls = path_to_save +  base_name_xls + y + "_woj_" +w + "_step_" + str(step)  + now_file + ".csv"
                         
                
                if step == rec_no:
                    finished = True
                    
                    name_xls_total = path_to_save + base_name_xls + y + "_woj_" +w + "_total_" + now_file + ".csv"
                    cars_in_years_and_woj.to_csv(path_or_buf= name_xls_total)
                    #del cars_in_years_and_woj
                    
                    # Agregation of the WOJ - all years in WOJ
                    # Agregation data - all years in one Woj
                    #print ("y=",y,"years[0]=",years[0])
                    if y == years[0]:
                        all_cars_in_years = cars_in_years_and_woj
                        #print ("!!!!!! first stp i all_cars_in_years = cars_in_years_and_woj")
                    else:
                        # add another set of data form steps in the common df
                        all_cars_in_years = all_cars_in_years.append(cars_in_years_and_woj)

                    
                else:    
                    step = step + 1
                
               
                
                print ("-"*5)
    
    
    
    name_xls_all_years = path_to_merge + base_name_xls +" WOJ-" + w + "_years_" + years[0]  + "-" + years[-1] + "_total_" +  now_file + ".csv"
    print ("saving woj:",w," to the path: ", name_xls_all_years)
    all_cars_in_years.to_csv(path_or_buf= name_xls_all_years)
    del cars_in_years_and_woj
    
    # end of WOj loop

----------------------------------------
wojedzództwo: 02
First year to dig: 2021 Last year to dig: 2022
J:/Dokumenty/Tomek/YT/samochody/cepik/grab/2022-06-05//woj_02/2021/  successfully created
--------------------
Year: 2021
no data to del
We dig in: https://api.cepik.gov.pl/pojazdy?&limit=500&data-od=20210101&data-do=20211231&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true&wojewodztwo=02&page=1
digging OK - data statatus code: 200
page/step:  1  Records in woj:  02 records to download: 952  pages to download:  3.0
-----
We dig in: https://api.cepik.gov.pl/pojazdy?&limit=500&data-od=20210101&data-do=20211231&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true&wojewodztwo=02&page=2
digging OK - data statatus code: 200
page/step:  2  Records in woj:  02 records to download: 952  pages to download:  3.0
-----
We dig in: https://api.cepik.gov.pl/pojazdy?&limit=500&data-od=20210101&data-do=20211231&f


Merging Files

In [9]:

print("Path with files to merge: ",path_to_merge)

files = [file for file in os.listdir(path_to_merge) if not file.startswith('.')] # Ignore hidden files
file_name = "all_data.csv"
all_data = pd.DataFrame()

for file in files:
    current_data = pd.read_csv(path_to_merge +"/"+file)
    all_data = pd.concat([all_data, current_data])
    
all_data.to_csv(path_to_merge + all_merged_data_file_name , index=False)
print("all merged data was saved to ", path_to_merge + all_merged_data_file_name)


Path with files to merge:  J:/Dokumenty/Tomek/YT/samochody/cepik/grab/2022-06-05/all/
all merged data was saved to  J:/Dokumenty/Tomek/YT/samochody/cepik/grab/2022-06-05/all/Cepik_all_data_in_one_file.csv
